## Семинар 4 

Всем привет и добро пожаловать на наш семинар по сбору новостей. Сегодня мы ставим себе следующие цели:

1. Собрать все заголовки статей на определенную тему, отфильтровав по дате с сайта делового издания "Коммерсант"
2. Узнать про хитрости, которые могут помочь в сборе данных
3. Провести самостоятельную практику с похожими заданиями

Для этого мы воспользуемся уже знакомыми Вам библиотеками requests и BeautifulSoup, а также познакомимся с новой – Selenium.

*Материал подготовила Анастасия Максимовская, хитрости позаимствованы у Филиппа Ульянкина. Вопросы по материалам можно писать мне в телеграм @anastasiyamaxx*

<img src="https://avatars.mds.yandex.net/get-zen_doc/3892121/pub_5f7ab7fb8d3ae5589bb054aa_5f7ab85061e6d41ef5615d94/scale_1200" width=700>

## Забираем заголовки с Коммерсанта

In [1]:
import requests 
from bs4 import BeautifulSoup

Итак, начнем с простого – проверим, соберется ли у нас информация с главной страницы Коммерсанта или нужно искать специальные примочки.

In [2]:
url = 'https://www.kommersant.ru/'
response = requests.get(url)
response

<Response [200]>

`<Response [200]>` выглядит хорошо. Но имейте в виду – это не всегда значит, что мы получили нужную информацию. Например, когда я пишу этот семинар, главная страница выглядит так:


<img src='imgs/pic1.png' width=800>

Однако, если бы нам вылетел баннер (например, какое-нибудь предложение о скидке) или запрос в духе "уточните Ваше местоположение", или капча, то некоторый нужный текст с главной страницы в собранный html мог бы не попасть. Для этого можно либо глазами посмотреть на `response.content` или попробовать найти нужный элемент с помощью методов `.find()` (находит первый элемент, отвечающий условиям в скобочках) или `.find_all()` (находит все нужные элементы) из библиотеки `bs4`.

Сделаем деревце, по которому можно искать нужные элементы с помощью `bs4`:

In [3]:
tree = BeautifulSoup(response.content, 'html.parser')

Найдем главную новость в тексте. Для этого я перехожу на сайт Коммерсанта, навожу мышкой на заголовок "Роспотребнадзор сократил до суток максимальный срок выполнения исследования на коронавирус", щелкаю правой кнопкой мыши и нажимаю "Просмотреть код элемента". Вижу что-то такое:

<img src="imgs/pic2.png">

Попробуем найти этот элемент в нашем дереве!

In [4]:
tree.find_all('a', {'class': 'top_news_main__link link'})

[<a class="top_news_main__link link" href="/doc/5038665?from=top_main_1">
 <span class="vicon vicon--hot_news top_news_main__lightning">
 <svg class="vicon__body"><use xlink:href="#vicon-hot_news" xmlns:xlink="http://www.w3.org/1999/xlink"></use></svg>
 </span>
                         В России заболеваемость и смертность от коронавируса второй день подряд обновляют максимумы — 33 208 заболевших, 1 002 умерших
                     </a>]

Достанем только текст:

In [8]:
tree.find('a', {'class': 'top_news_main__link link'}).text.strip()

'В России заболеваемость и смертность от коронавируса второй день подряд обновляют максимумы — 33 208 заболевших, 1 002 умерших'

Однако, если Вы впервые заходите на сайт или откроете окно в режиме инкогните, то увидите, что при первом визите на сайт вылетает такой баннер:

<img src="imgs/pic3.png">

Также это можно заметить, полистав содержимое `tree`.

Конкретно в этом примере нам это не мешает вытащить заголовок. Однако, иногда такие всплывающие окна мешют собрать html с нужной информацией со страницы. Что же делать? Нам на помощь придет библиотека selenium – специальный инструмент для автоматизации действий браузера.

### Добавляем селениум :)

Библиотека `selenium` – набор инструментов для интерактивной работы в браузере средствами Python. Вообще Selenium ‒ это целый проект, в котором есть разные инструменты. Мы рассмотрим один из самых распространенных ‒ Selenium WebDriver, модуль, который позволяется Python встраиваться в браузер и работать в нем как пользователь: кликать на ссылки и кнопки, заполнять формы, выбирать опции в меню и прочее. 

In [9]:
# через восклицательный знак обращемся к командной строке (на маке называется terminal)
# pip – менеджер пакетов для питона, который позволяет нам поставить библиотеку
!pip install selenium
!pip install webdriver-manager

Для того, чтобы воспользоваться библиотекой, нужно загрузить вебдрайвер для Вашего браузера. Подробнее можно почитать [в пункте 1.5 документации про установку](https://selenium-python.readthedocs.io/installation.html). План действий такой: качате драйвер – прописываете путь в переменной PATH – используете.

Но мы воспользуемся лайфхаком, чтобы не мучиться долго с установкой. Это библиотека `webdriver-manager`, которая скачает вебдрайвер за Вас. Подробнее в [документации](https://pypi.org/project/webdriver-manager/) (там же можно посмотреть код для других браузеров).

In [10]:
from selenium.webdriver.common.keys import Keys

In [11]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_mac64.zip
Driver has been saved in cache [/Users/a18509896/.wdm/drivers/chromedriver/mac64/94.0.4606.61]


На Вашем компьютере откроется пустая страничка. Давайте перейдем на сайт Коммерсанта.

In [12]:
driver.get('https://www.kommersant.ru/')

Откройте эту страничку – теперь она не пустая :) 

Следующим шагом, нам надо понять, как кликнуть на баннер так, чтобы он закрылся. Для этого нужно определить пусть к кнопке. Как и раньше, наводим мышку и кликаем "Просмотреть код".

<img src="imgs/pic4.png">

Теперь нужно сделать 2 действия кодом:

1. Помочь драйверу найти элемент
2. Кликнуть на него

Есть несколько способов указать пусть к элементу, они описаны [здесь](https://selenium-python.readthedocs.io/locating-elements.html) (попросите Вашего семинариста вкратце прокомментировать каждый). 

Принципиальной разницы нам сейчас нет, предлагаю воспользоваться методом `driver.find_element_by_css_selector()`. Правой кнопокой мыши щелкните по коду нужной кнопки (принять или отклонить), выберите copy – copy selector.

<img src="imgs/pic5.png" width=500>

Сохраним селектор в переменную, найдем нужный элемент и кликнем. Иногда работает не с первого раза.

In [13]:
selector = "body > div.subscription-popup-v2.subscription-popup-v2_push-popup > div.subscription-popup-v2__inner-container > div.subscription-popup-v2__controls > div.subscription-popup-v2__reject"

In [14]:
ss = driver.find_elements_by_css_selector(selector)[0]

In [15]:
ss

<selenium.webdriver.remote.webelement.WebElement (session="3a8cc8da6e7ae8d2d55b495676e1a6b0", element="5432e9cc-ba33-4c6b-8cea-71f3a37c1c16")>

In [16]:
ss.click()

Обновим страничку на всякий случай:

In [17]:
driver.refresh()

Давайте найдем главный заголовок еще одним способом. Сначала найдем элемент, помня имя класса (см. скрины выше), потом достанем его html код.

In [18]:
main_news = driver.find_element_by_class_name("top_news_main__name")
main_news

<selenium.webdriver.remote.webelement.WebElement (session="3a8cc8da6e7ae8d2d55b495676e1a6b0", element="334f3b1b-8f19-473e-a634-54f96a97c663")>

In [19]:
main_news.get_attribute('innerHTML')

'\n                    <a href="/doc/5038665?from=top_main_1" class="top_news_main__link link">\n                        <span class="vicon vicon--hot_news top_news_main__lightning">\n                            <svg class="vicon__body"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="#vicon-hot_news"></use></svg>\n                        </span>\n                        В России заболеваемость и смертность от коронавируса обновили максимумы — 33 208 заболевших, 1 002 умерших\n                    </a>\n                '

In [20]:
small_tree = BeautifulSoup(main_news.get_attribute('innerHTML'), 'html.parser')

In [21]:
small_tree


<a class="top_news_main__link link" href="/doc/5038665?from=top_main_1">
<span class="vicon vicon--hot_news top_news_main__lightning">
<svg class="vicon__body"><use xlink:href="#vicon-hot_news" xmlns:xlink="http://www.w3.org/1999/xlink"></use></svg>
</span>
                        В России заболеваемость и смертность от коронавируса обновили максимумы — 33 208 заболевших, 1 002 умерших
                    </a>

In [22]:
small_tree.text.strip()

'В России заболеваемость и смертность от коронавируса обновили максимумы — 33 208 заболевших, 1 002 умерших'

Ура, получили заголовок главной новости. Если он отличается от того, что на скрине, то нестрашно – новостные сайты быстро меняют статьи на главных страницах. Остальное можно достать по аналогии :)

Перейдем к более интересному – соберем все новости на определенную тему и срок.

Предлагаю попробовать собрать все новости, содержащие фразу "центральный банк" за период с 24 августа 2021 по текущий день. То есть, переводя это на программистский, нам нужно проделать следующие действия:

1. Найти окно поиска, кликнуть
2. Ввести в него ключевую фразу, нажать кнопку поиска
3. Нажать кнопку расширенный поиск
4. Найти кнопку, где изменяем дату начала поиска, выставить нужную нам
5. Собрать информацию

Давайте начнем :) В прошлый раз мы воспользовались поиском с помощью селектора `.find_element_by_css_selector()`. Теперь добавим немного разнообразия и сделаем поиском через XPath. Получить ее можно по старой схеме: наводим мышь на окно поиска – кликаем посмотреть код – правой кнопкой кликаем по мыши на выделенном коде – выбираем copy – copy xpath.   

По шагам:

1. наводим мышь на окно поиска – кликаем посмотреть код
<img src="imgs/pic6.png" width=800 alt="aa"> 
2. правой кнопкой мыши кликаем на выделенном коде – выбираем copy – copy xpath   
<img src="imgs/pic7.png" width=500> 

In [24]:
"Гарик "Бульдог" Харламов"

SyntaxError: invalid syntax (<ipython-input-24-1846875b534c>, line 1)

In [25]:
"Гарик \"Бульдог\" Харламов"

'Гарик "Бульдог" Харламов'

In [26]:
'Гарик "Бульдог" Харламов'

'Гарик "Бульдог" Харламов'

In [34]:
# найденный по инструкции выше xpath к лупе
xpath_query = '//*[@id="js-navsearch-submit"]'
# находим окно поиска
search = driver.find_element_by_xpath(xpath_query)
# кликаем на него
search.click()

In [35]:
# найденный по инструкции выше xpath к окну поиска
xpath_query = '//*[@id="js-navsearch-query"]'
# находим окно поиска
search = driver.find_element_by_xpath(xpath_query)
# кликаем на него
search.click()

In [36]:
search_term = "центральный банк"
# печатаем фразу для поиска в окне для поиска
search.send_keys(search_term)
# нажимаем кнопку enter
search.send_keys(Keys.RETURN)

Если Вы посмотрите, что происходит в окне браузера, которым управляет селениум, то увидите, что окно поменялось и мы теперь в разделе поиска :) 

<img src="imgs/pic8.png" width=500>

Нажимаем на кнопку расширенный поиск и выбираем дату.

Дальше мы все уже знаем. Откройте в соседней с ноутбуком вкладке сайт коммерсанта и доставайте оттуда нужные селекторы / xpath (неважно).

In [37]:
# находим селектор для кнопки расширенный поиск и нажимаем ее
selector2 = "body > main > div > div > section > div.grid-col.grid-col-s3 > form > div.ui-field_pack > label"


In [38]:
ext_search = driver.find_element_by_css_selector(selector2)

In [39]:
ext_search.click()

In [40]:
# находим селектор для поля ввода даты
selector3 = "body > main > div > div > section > div.grid-col.grid-col-s3 > form > div.ui-collapse.js-toggle-collapse.js-toggle-item.ui-collapse--show > section.simple_page_section.simple_page_section--form.js-search-settings > div.grid.ui-collapse.js-toggle-collapse.js-toggle-item.ui-collapse--show > div:nth-child(1) > div > input"


In [41]:
date = driver.find_element_by_css_selector(selector3)

Обратите внимание на картинку ниже – дата начала поиска по дефолту вбита в окошко, надо ее удалить.

<img src="imgs/pic9.png" width=500>

In [42]:
# удаляем введеный по дефолту текст в ячейке
date.send_keys(Keys.SHIFT, Keys.END, Keys.BACK_SPACE)

In [43]:
# вводим нужную дату и надижимаем enter
date_start = "24.08.2021"
date.send_keys(date_start)

In [44]:
date.send_keys(Keys.RETURN)

Ура, получили нужную выдачу! Попробуем перейти на следующую страничку.

In [45]:
# путь к кнопке следующая страница
xpath3 = "/html/body/main/div/div/section/div[1]/div[3]/a"

In [46]:
second_page = driver.find_element_by_xpath(xpath3)

In [47]:
second_page.click()

Посмотрим на адрес нашей странички:

In [48]:
driver.current_url

'https://www.kommersant.ru/search/results?search_query=%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9+%D0%B1%D0%B0%D0%BD%D0%BA&sort_type=1&search_full=1&time_range=2&dateStart=2021-08-24&dateEnd=2021-10-16&page=2'

In [50]:
# driver.page_source

Обратите внимание на параметр `page=2`. Если мы будем менять номера, то будем перемещаться по всем страницам с заголовками, удовлетворяющим нашим условиям. Осталось написать функцию, которая будет доставать нужную информацию с одной странички, и запустить ее циклом для всех.

Начнем с того, как задать url. Обратите внимание на обратный слэш, это так называемый line continuation character. Он означает, что код продолжится на следующей строке. Также обратите внимание на букву f перед продолжением url-адреса на 3 строчке – она позваоляет мне подставить значение переменной `{page_num}` в середину строки.

In [51]:
page_num = 1
url = 'https://www.kommersant.ru/search/results?search_query='\
    '%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9+%D0%B1'\
    '%D0%B0%D0%BD%D0%BA&sort_type=1&search_full=1&time_range=2&'\
    f'dateStart=2021-08-24&dateEnd=2021-10-15&page={page_num}'

In [52]:
url

'https://www.kommersant.ru/search/results?search_query=%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9+%D0%B1%D0%B0%D0%BD%D0%BA&sort_type=1&search_full=1&time_range=2&dateStart=2021-08-24&dateEnd=2021-10-15&page=1'

Как обычно забираем HTML-разметку и делаем деревце бьютифул супом.

In [53]:
response2 = requests.get(url)
response2

<Response [200]>

In [54]:
tree_search = BeautifulSoup(response2.content, 'html.parser')

Уже знакомый по лекциям механизм поиска элемента по html разметке.

In [55]:
# находим заголовки
headers = tree_search.find_all('h2', {'class': 'uho__name rubric_lenta__item_name'})

In [57]:
len(headers)

10

In [58]:
headers[0]

<h2 class="uho__name rubric_lenta__item_name">
<a class="uho__link uho__link--overlay" href="/doc/5007292?query=%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9%20%D0%B1%D0%B0%D0%BD%D0%BA" target="_blank">«Одна из основных проблем — монетарная политика <mark>центральных</mark> <mark>банков</mark>»</a>
</h2>

Если достать из тега текст, то можно заметить, что есть пробелы / переходы на новые строки в начале и конце. Метод `.strip()` избавится от них.

In [59]:
headers[0].text

'\n«Одна из основных проблем — монетарная политика центральных банков»\n'

In [60]:
headers[0].text.strip()

'«Одна из основных проблем — монетарная политика центральных банков»'

In [61]:
# находим подзаголовки
subheaders = tree_search.find_all('h3', \
                {'class': 'uho__subtitle rubric_lenta__item_subtitle'})

In [62]:
len(subheaders)

6

Подзаголовки есть не у всех новостей!

In [63]:
subheaders[0]

<h3 class="uho__subtitle rubric_lenta__item_subtitle">
<a class="uho__link" href="/doc/5007292?query=%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9%20%D0%B1%D0%B0%D0%BD%D0%BA" target="_blank">Олег Богданов — о причинах роста стоимости газа в Европе</a>
</h3>

In [64]:
subheaders[0].text

'\nОлег Богданов — о причинах роста стоимости газа в Европе\n'

In [65]:
subheaders[0].text.strip()

'Олег Богданов — о причинах роста стоимости газа в Европе'

In [66]:
# напишем функцию, которая будет выдавать список из словарей 
# в каждом словаре заголовок и описание
def get_page_info(page_num):
    url = 'https://www.kommersant.ru/search/results?search_query='\
        '%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9+%D0%B1'\
        '%D0%B0%D0%BD%D0%BA&sort_type=1&search_full=1&time_range=2&'\
        f'dateStart=2021-08-24&dateEnd=2021-10-15&page={page_num}'
    
    response = requests.get(url)
    tree_search = BeautifulSoup(response.content, 'html.parser')
    headers = tree_search.find_all('h2', \
                                   {'class': 'uho__name rubric_lenta__item_name'})
    subheaders = tree_search.find_all('h3', \
                            {'class': 'uho__subtitle rubric_lenta__item_subtitle'})
    result = []
    for i in range(len(headers)):
        header = headers[i]
        try:
            subheader = subheaders[i]
            d = {'article_header': header.text.strip(),
                 'article_subheader': subheader.text.strip()}
        except:
            d = {'article_header': header.text.strip(),
                 'article_subheader': ''}
        result.append(d)
    return result

In [67]:
all_data = []
for n in range(1, 14):
    all_data.extend(get_page_info(n))

In [68]:
len(all_data)

130

Пока не очень знакомая библиотека сделает табличку из списка и позволит сохранить ее в файл.

In [69]:
import pandas as pd

In [70]:
df = pd.DataFrame(all_data)

In [71]:
df.head()

,article_header,article_subheader
0,«Одна из основных проблем — монетарная политик...,Олег Богданов — о причинах роста стоимости газ...
1,«Платина» решила взять ЦБ силой,Ксения Дементьева о методах борьбы банков за в...
2,Банк «Открытие» планирует размещение трехлетни...,Объем нового выпуска серии БО-П09 составит не ...
3,Пункты плана подготовки Банком России обзора ДКП,Как к нему подготовиться и как пережить
4,Банк Норвегии первым из крупных западных ЦБ по...,Чего ожидать инвесторам и вкладчикам от повыше...


In [235]:
# сохранить в csv формат
# index=False сделает так, чтобы колонка с индексами не вогла в итоговый файл
df.to_csv('all_data.csv', index=False)

In [72]:
# сохранить в xlsx формат
df.to_excel('all_data.xlsx', index=False)

In [73]:
# не забываем закрыть браузер драйвером после завершения работы :)
driver.close()

## Что делать, если сервер разозлился?

* Вы решили собрать себе немного данных 
* Сервер не в восторге от ковровой бомбардировки автоматическими запросами 
* Error 403, 404, 504, $\ldots$ 
* Капча, требования зарегистрироваться
* Заботливые сообщения, что с вашего устройства обнаружен подозрительный трафик

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/doge.jpg" width="450"> 

### а) быть терпеливым 

* Слишком частые запросы раздражают сервер
* Ставьте между ними временные задержки 

In [74]:
import time
time.sleep(3) # и пусть весь мир подождёт 3 секунды

### б) быть похожим на человека


Запрос нормального человека через браузер выглядит так: 

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/browser_get.png" width="600"> 
    
С ним на сервер попадает куча информации! Запрос от питона выглядит так: 


<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/python_get.jpg" width="250"> 
 
Заметили разницу?  Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными и пустить пыль в глаза сервера при помощи генерации фейкового юзер-агента. Библиотек, которые справляются с такой задачей, существует очень и очень много, лично мне больше всего нравится [fake-useragent.](https://pypi.org/project/fake-useragent/) При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [19]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (X11; CrOS i686 4319.74.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.57 Safari/537.36'

Например, https://knowyourmeme.com/ не захочет пускать к себе python и выдаст ошибку 403. Она выдается сервером, если он доступен и способен обрабатывать запросы, но по некоторым личным причинам отказывается это делать.

In [20]:
url = 'https://knowyourmeme.com/'

response = requests.get(url)
response

<Response [403]>

А если сгенерировать User-Agent, вопросов у сервера не возникнет. 

In [21]:
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

__Другой пример:__ если захотите спарсить ЦИАН, он начнет вам выдавать капчу. Один из вариантов обхода: менять ip через тор. Однако на практически каждый запрос из-под тора, ЦИАН будет выдавать капчу. Если добавить в запрос `User_Agent`, то капча будет вылезать намного реже. 

### в) общаться через посредников

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/proxy.jpeg" width="400"> 

Посмотрим на свой ip-адрес без прокси. 

In [22]:
r = requests.get('https://httpbin.org/ip')
print(r.json())

{'origin': '195.208.27.151, 195.208.27.151'}


А теперь попробуем посмотреть, что будет если подключить прокси.

In [24]:
proxies = {
    'http': '182.53.206.47:47592',
    'https': '182.53.206.47:47592'
}

r = requests.get('https://httpbin.org/ip', proxies=proxies)

print(r.json())

{'origin': '182.53.206.47, 182.53.206.47'}


Запрос работал немного подольше, ip адрес сменился. Большая часть проксей, которые вы найдёте работают криво. Иногда запрос идёт очень долго и выгоднее сбросить его и попробовать другую проксю. Это можно настроить опцией `timeout`.  Например, так если сервер не будет отвечать секунду, код упадёт. 

In [27]:
import requests
requests.get('http://www.google.com', timeout=1)

<Response [200]>

У requests есть довольно много разных интересных примочек. Посмотреть на них можно в [гайде из документации.](https://requests.readthedocs.io/en/master/user/advanced/)


__Где можно попытаться раздобыть списки прокси:__ 

* https://qna.habr.com/q/591069
* https://getfreeproxylists.blogspot.com/
* Большая часть бесплатных прокси обычно не работает. Пишите парсер, который будет собирать списки из проксей и пытаться применить их. 

### г) уходить глубже 

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/tor.jpg" width="600"> 

Можно попытаться обходить злые сервера через тор. Есть аж несколько способов, но мы про это говорить не будем. Лучше подробно почитать в нашей статье на Хабре. Ссылка на неё в конце тетрадки. Ещё в самом начале была. А ещё в середине [наверняка есть.](https://habr.com/ru/company/ods/blog/346632/)

### Совместить всё? 

1. Начните с малого 
2. Если продолжает банить, накидывайте новые примочки
3. Каждая новая примочка бьёт по скорости 
4. [Разные примочки для requests](http://docs.python-requests.org/en/v0.10.6/user/advanced/)

## Хитрости для парсинга :)

### Хитрость 1:  Не стесняйтесь пользоваться `try-except`

Эта конструкция позволяет питону в случае ошибки сделать что-нибудь другое либо проигнорировать её. Например, мы хотим найти логарифм от всех чисел из списка: 

In [50]:
from math import log 

a = [1,2,3,-1,-5,10,3]

for item in a:
    print(log(item))

0.0
0.6931471805599453
1.0986122886681098


ValueError: math domain error

У нас не выходит, так как логарифм от отрицательных чисел не берётся. Чтобы код не падал при возникновении ошибки, мы можем его немного изменить: 

In [51]:
from math import log 

a = [1,2,3,-1,-5,10,3]

for item in a:
    try:
        print(log(item))  # попробуй взять логарифм
    except:
        print('я не смог') # если не вышло, сознайся и работай дальше

0.0
0.6931471805599453
1.0986122886681098
я не смог
я не смог
2.302585092994046
1.0986122886681098


__Как это использовать при парсинге?__  Интернет создаёт человек. У многих людей руки очень кривые. Предположим, что мы на ночь поставили парсер скачивать цены, он отработал час и упал из-за того, что на како-нибудь одной странице были криво проставлены теги, либо вылезло какое-то редкое поле, либо вылезли какие-то артефакты от старой версии сайта, которые не были учтены в нашем парсере. Гораздо лучше, чтобы код проигнорировал эту ошибку и продолжил работать дальше. 

### Хитрость 2:  pd.read_html

Если на странице, которую вы спарсили, среди тэгов `<tr>` и `<td>` прячется таблица, чаще всего можно забрать её себе без написания цикла, который будет перебирать все стобцы и строки. Поможет в этом `pd.read_html`. Например, вот так можно забрать себе [табличку с сайта ЦБ](https://cbr.ru/currency_base/daily/) 

In [52]:
import pandas as pd

df = pd.read_html('https://cbr.ru/currency_base/daily/', header=-1)[0]
df.head()

,Цифр. код,Букв. код,Единиц,Валюта,Курс
0,36,AUD,1,Австралийский доллар,421246
1,944,AZN,1,Азербайджанский манат,361115
2,51,AMD,100,Армянских драмов,127965
3,933,BYN,1,Белорусский рубль,288800
4,975,BGN,1,Болгарский лев,347750


Команда пытается собрать в массив все таблички c веб-страницы. Если хочется, можно сначала через bs4 найти нужную таблицу, а потом уже распарсить её: 

In [53]:
resp = requests.get('https://cbr.ru/currency_base/daily/')
tree = BeautifulSoup(resp.content, 'html.parser')

# нашли табличку
table = tree.find_all('table', {'class' : 'data'})[0]

# распарсили её
df = pd.read_html(str(table), header=-1)[0]
df.head()

,Цифр. код,Букв. код,Единиц,Валюта,Курс
0,36,AUD,1,Австралийский доллар,421246
1,944,AZN,1,Азербайджанский манат,361115
2,51,AMD,100,Армянских драмов,127965
3,933,BYN,1,Белорусский рубль,288800
4,975,BGN,1,Болгарский лев,347750


### Хитрость 3:  используйте пакет tqdm

> Код уже работает час. Я вообще без понятия когда он закончит работу. Было бы круто узнать, сколько ещё ждать... 

Если в вашей голове возникла такая мысль, пакет `tqdm` ваш лучший друг. Установите его: ```pip install tqdm```

In [75]:
from tqdm import tqdm_notebook

a = list(range(30))

# 30 раз будем спать по секунде
for i in tqdm_notebook(a):
    time.sleep(1)

<ipython-input-75-b83b2df6bc34>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(a):


Мы обмотали тот вектор, по которому идёт цикл в `tqdm_notebook`. Это даёт нам красивую зелёную строку, которая показывает насколько сильно мы продвинулись по коду. Обматывайте свои самые большие и долгие циклы в `tqdm_notebook` и всегда понимайте сколько осталось до конца. 

### Хитрость 4:  распаралеливание

Если сервер не очень настроен вас банить, можно распаралелить свои запросы к нему. Самый простой способ сделать это — библиотека `joblib`. 

In [55]:
from joblib import Parallel, delayed
from tqdm import tqdm_notebook

def simple_function(x):
    return x**2


nj = -1 # паралель на все ядра 
result = Parallel(n_jobs=nj)(
                delayed(simple_function)(item)          # какую функцию применяем 
                for item in tqdm_notebook(range(10)))   # к каким объектам применям

# tqdm_notebook в последней строчке будет создавать зелёный бегунок с прогрессом

На самом деле это не самый эффективный способ паралелить в python. Он жрёт много памяти и работает медленнее, чем [стандартный multiprocessing.](https://docs.python.org/3/library/multiprocessing.html) Но зато две строчки, КАРЛ! Две строчки! 

### Хитрость 5:  selenium без браузера

Селениум можно настроить так, чтобы физически браузер не открывался.

In [56]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

options = Options()
options.headless = True 
driver = webdriver.Firefox(options=options)

ref = 'http://google.com'
driver.get(ref)

driver.close()

###  Ещё хитрости: 

* __Сохраняйте то, что парсите по мере скачки!__ Прямо внутрь цикла запихните код, который сохраняет файл! 
* Когда код упал в середине списка для скачки, не обязательно запускать его с самого начала. Просто сохраните тот кусок, который уже скачался и дозапустите код с места падения.
* Засовывать цикл для обхода ссылок внутрь функции - не самая хорошая идея. Предположим, что надо обойти $100$ ссылок. Функция должна вернуть на выход объекты, которые скачались по всему этому добру. Она берёт и падает на $50$ объекте. Конечно же то, что уже было скачано, функция не возвращает. Всё, что вы накачали - вы теряете. Надо запускать заново. Почему? Потому что внутри функции своё пространство имён. Если бы вы делали это циклом влоб, то можно было бы сохранить первые $50$ объектов, которые уже лежат внутри листа, а потом продолжить скачку. 
* Можно ориентироваться на html-страничке с помощью `xpath`. Он предназначен для того, чтобы внутри html-странички можно было быстро находить какие-то элементы. [Подробнее можно почитать тут.](https://devhints.io/xpath)
* Не ленитесь листать документацию. Из неё можно узнать много полезных штук. 

### Почиташки

* [Парсим мемы в python](https://habr.com/ru/company/ods/blog/346632/) - подробная статья на Хабре, по которой можно научиться ... парсить (ВНЕЗАПНО) 
* [Тетрадки Ильи Щурова](https://github.com/ischurov/pythonhse) про python для анализа данных. В [лекции 9](https://nbviewer.jupyter.org/github/ischurov/pythonhse/blob/master/Lecture%209.ipynb) и [лекции 10](https://nbviewer.jupyter.org/github/ischurov/pythonhse/blob/master/Lecture%2010.ipynb) есть про парсеры. 
* [Книга про парсинг](https://github.com/FUlyankin/Parsers/blob/master/Ryan_Mitchell_Web_Scraping_with_Python-_Collecting_Data_from_the_Modern_Web_2015.pdf) на случай если вам совсем скучно и хочется почитать что-то длинное и на английском
* [Продвинутое использование requests](https://2.python-requests.org/en/master/user/advanced/)
* [Перевод документации по selenium на русский на хабре](https://habr.com/ru/post/248559/)


* [Страничка с парсерами Филиппа Ульянкина](https://fulyankin.github.io/Parsers/) (на материалах которого основан список лайфхаков – спасибо!): 
    * [Более подробно про selenium](https://nbviewer.jupyter.org/github/FUlyankin/Parsers/blob/master/sems/3_Selenium_and_Tor/4.1%20Selenium%20.ipynb)
    * [Немного устаревший гайд по парсинг вконтакте](https://nbviewer.jupyter.org/github/FUlyankin/ekanam_grand_research/blob/master/0.%20vk_parser_tutorial.ipynb)
    * [Немного устаревший гайд про google maps](https://nbviewer.jupyter.org/github/FUlyankin/Parsers/blob/master/Parsers%20/Google_maps_API.ipynb)



## Практика

Основная цель практики – убедиться, что Вам понятен код с семинара. Попробуйте собрать информацию по аналогии с сайта Коммерсанта. Фразу для поиска оставляем на Ваш выбор, срок – за последний месяц.